In [1]:
!pip install torchsummary

In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder, DatasetFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from glob import glob
import gc
import sys
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchsummary

if torch.cuda.is_available():
    device = torch.device("cuda")  # using first available GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Switching to CPU.")

GPU is not available. Switching to CPU.


In [3]:
# show images and labels
def show_images(images, labels):
    plt.figure(figsize=(12, 6))
    for i in range(len(images)):
        plt.subplot(4, 8, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))
        plt.title(labels[labels[i]])
        plt.axis('off')
    plt.show()

In [4]:
# define img data transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [5]:
batch_size = 30
# load preparing dataset
trainDataset = ImageFolder(root='/kaggle/input/7015-dataset/train/train', transform=transform)
trainLoader = DataLoader(trainDataset, batch_size=batch_size, shuffle=False)
testDataset = ImageFolder(root='/kaggle/input/7015-dataset/test/test', transform=transform)
testLoader = DataLoader(testDataset, batch_size=batch_size, shuffle=False)
labels = trainDataset.classes

In [6]:
train_samples = torch.tensor([]).to(device)
train_labels = torch.tensor([]).to(device)
test_samples = torch.tensor([]).to(device)
test_labels = torch.tensor([]).to(device)

In [7]:
for images, labels in tqdm(trainLoader):
    images = images.to(device)
    images = (images * 255).to(torch.uint8)
    train_samples = torch.cat([train_samples, images.unsqueeze(0)], dim=0)
    label = list(map(lambda x: x.split('_')[1], trainDataset.class_to_idx.keys()))[labels[0]]
    if label == 'arm':
        label = 0
    else:
        label = 1
    train_labels = torch.cat([train_labels, torch.tensor(label).unsqueeze(0).to(device)], dim=0)
for images, labels in tqdm(testLoader):
    images = images.to(device)
    images = (images * 255).to(torch.uint8)
    test_samples = torch.cat([test_samples, images.unsqueeze(0)], dim=0)
    label = list(map(lambda x: x.split('_')[1], testDataset.class_to_idx.keys()))[labels[0]]
    if label == 'arm':
        label = 0
    else:
        label = 1
    test_labels = torch.cat([test_labels, torch.tensor(label).unsqueeze(0).to(device)], dim=0)


100%|██████████| 109/109 [01:39<00:00,  1.10it/s]


In [8]:
train_samples = train_samples.to('cpu')
test_samples = test_samples.to('cpu')

In [8]:
# clear the unusage cache
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

AttributeError: module 'torch._C' has no attribute '_cuda_resetPeakMemoryStats'

In [10]:
class simple3dConv(nn.Module):
    def __init__(self, num_classes):
        super(simple3dConv, self).__init__()
        self.net = nn.Sequential(
        nn.Conv3d(3, 64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool3d(2),
        nn.BatchNorm3d(64),

#         nn.Conv3d(64, 64, kernel_size=3),
#         nn.ReLU(),
#         nn.MaxPool3d(2),
#         nn.BatchNorm3d(64),

        nn.Conv3d(64, 128, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool3d(2),
        nn.BatchNorm3d(128),

        nn.Conv3d(128, 256, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool3d(2),
        nn.BatchNorm3d(256),
            
        nn.AdaptiveAvgPool3d((1,1,1)),
            
        nn.Flatten(),
        nn.Linear(256, 512),
        nn.ReLU(),
        
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
        )
    def forward(self, x):
        x = self.net(x)
        return x
    
class myDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

In [11]:
CNN_batch_size = 8
traindataset3d = myDataset(train_samples, train_labels)
traindataloader3d = DataLoader(traindataset3d, batch_size=CNN_batch_size, shuffle=True)
testdataset3d = myDataset(test_samples, test_labels)
testdataloader3d = DataLoader(testdataset3d, batch_size=CNN_batch_size, shuffle=False)

In [12]:
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim.lr_scheduler import CosineAnnealingLR

initial_learning_rate = 3e-5
# decay_steps = 10
# decay_rate = 0.96
epochs = 80
num_classes = 2

model = simple3dConv(num_classes).to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=initial_learning_rate)

# lr_scheduler = ExponentialLR(optimizer, gamma=decay_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

In [13]:
import gc
del train_samples, test_samples
gc.collect()

211

In [14]:
torchsummary.summary(model, (3,30,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 64, 28, 222, 222]           5,248
              ReLU-2     [-1, 64, 28, 222, 222]               0
         MaxPool3d-3     [-1, 64, 14, 111, 111]               0
       BatchNorm3d-4     [-1, 64, 14, 111, 111]             128
            Conv3d-5    [-1, 128, 12, 109, 109]         221,312
              ReLU-6    [-1, 128, 12, 109, 109]               0
         MaxPool3d-7       [-1, 128, 6, 54, 54]               0
       BatchNorm3d-8       [-1, 128, 6, 54, 54]             256
            Conv3d-9       [-1, 256, 4, 52, 52]         884,992
             ReLU-10       [-1, 256, 4, 52, 52]               0
        MaxPool3d-11       [-1, 256, 2, 26, 26]               0
      BatchNorm3d-12       [-1, 256, 2, 26, 26]             512
AdaptiveAvgPool3d-13         [-1, 256, 1, 1, 1]               0
          Flatten-14                  [

In [ ]:
import random
import os
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 使用这个函数来设置一个全局种子
seed_everything(42)

for epoch in range(epochs):
    model.train()
    train_correct = 0
    train_loss = []
    for inputs, labels in traindataloader3d:
        optimizer.zero_grad()

        inputs = torch.permute(inputs,(0, 2, 1, 3, 4))

        # 将数据输入模型
        outputs = model(inputs.to(device))
        
        train_correct += sum(torch.argmax(outputs, dim=1) == labels)
        # 计算损失
        batch_loss = criterion(outputs, labels.to(torch.long))
        train_loss.append(batch_loss)
        # 反向传播和优化
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
    model.eval()
    test_correct = 0
    test_loss = []
    with torch.no_grad():
        for inputs, labels in testdataloader3d:
            inputs = torch.permute(inputs,(0, 2, 1, 3, 4))
            outputs = model(inputs.to(device))
            predicted_labels = torch.argmax(outputs, dim=1)
            test_loss.append(criterion(outputs, labels.to(torch.long)))
            test_correct += sum(predicted_labels == labels)
            print(torch.argmax(outputs, dim=1), labels)

    print("training acc {:.5f}".format(train_correct / len(traindataloader3d.dataset.data)),
          "training loss {:.5f}".format(torch.mean(torch.tensor(train_loss)).item()),
          "----------------",
          "testing acc {:.5f}".format(test_correct / len(testdataloader3d.dataset.data)),
          "testing loss {:.5f}".format(torch.mean(torch.tensor(test_loss)).item()),
         )
    gc.collect()

tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 1, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 1, 1, 1, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 1, 1, 0, 1, 1, 1]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1, 1, 0, 1, 0, 1, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0, 0, 0, 0]) tensor([0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0]) tensor([0., 0.